<a href="https://www.kaggle.com/code/juancmoreno1508/red-neuronal?scriptVersionId=257149440" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
import time
import numpy as np

# Comprobar la disponibilidad de la GPU
print(f'GPU disponible: {tf.config.list_physical_devices("GPU")}')
device_name = tf.test.gpu_device_name()
print(f'Using device: {device_name if device_name else "CPU"}')

2025-08-20 18:14:34.904425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755713675.112119      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755713675.173275      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU disponible: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Using device: /device:GPU:0


I0000 00:00:1755713689.499069      19 gpu_device.cc:2022] Created device /device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [2]:
# 1. Preparación de los Datos
# Carga del conjunto de datos CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalizar los datos de píxeles (valores entre 0 y 1)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 17s 0us/step


In [3]:
# 2. Definición del Modelo
#-----------------------------------------------------------------------------------------------------------------
# Se define una Red Neuronal Convolucional (CNN) secuencial
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)),
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu', kernel_initializer='he_uniform'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1755713710.284890      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [4]:
# 3. Compilación del Modelo
#-----------------------------------------------------------------------------------------------------------------
# Se utiliza el optimizador Adam y la función de pérdida SparseCategoricalCrossentropy
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
# 4. Callback para Medir el Tiempo por Época
#-----------------------------------------------------------------------------------------------------------------
class TimeHistory(Callback):
    """Callback para medir y registrar el tiempo de cada época."""
    def on_train_begin(self, logs={}):
        self.times = []
    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_start_time = time.time()
    def on_epoch_end(self, epoch, logs={}):
        elapsed_time = time.time() - self.epoch_start_time
        self.times.append(elapsed_time)
        print(f"Epoca {epoch + 1} Terminada en {elapsed_time:.2f} Segundos")

time_callback = TimeHistory()

In [6]:
# 5. Entrenamiento del Modelo
#-----------------------------------------------------------------------------------------------------------------
print("Starting training...")
history = model.fit(x_train, y_train, epochs=16, batch_size=128,
                    validation_data=(x_test, y_test), callbacks=[time_callback],
                    verbose=1)

print('Finished Training')

Starting training...
Epoch 1/16


I0000 00:00:1755713716.028195      60 service.cc:148] XLA service 0x7851ec029350 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755713716.028810      60 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1755713716.373757      60 cuda_dnn.cc:529] Loaded cuDNN version 90300


 24/391 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0892 - loss: 2.7625

I0000 00:00:1755713720.209671      60 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2062 - loss: 2.1506Epoca 1 Terminada en 14.75 Segundos
391/391 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.2065 - loss: 2.1500 - val_accuracy: 0.4701 - val_loss: 1.4400
Epoch 2/16
389/391 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4508 - loss: 1.5146Epoca 2 Terminada en 3.00 Segundos
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.4510 - loss: 1.5142 - val_accuracy: 0.5780 - val_loss: 1.1807
Epoch 3/16
384/391 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5316 - loss: 1.3038Epoca 3 Terminada en 2.97 Segundos
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5318 - loss: 1.3032 - val_accuracy: 0.6268 - val_loss: 1.0640
Epoch 4/16
384/391 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5875 - loss: 1.1635Epoca 4 Terminada en 2.98 Segundos
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5877 - loss: 1.1631 - val_accuracy: 0.6520 - val_loss: 0.9868
Epoch 5/16
384/391 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accur

In [7]:
# 6. Evaluación del Modelo y Guardado
#-----------------------------------------------------------------------------------------------------------------
# Evaluar el modelo con el conjunto de prueba
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print(f'Precision de la red en el test de 1000 imagenes: {acc * 100:.2f} %')

# Guardar el modelo para su uso posterior en la arquitectura cliente-servidor
model.save('cifar_model.h5')

Precision de la red en el test de 1000 imagenes: 77.47 %
